In [ ]:
#| default_exp nb_01_graphs

In [ ]:
import pandas as pd
from imutils import contours
from skimage import measure
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
from skimage.future import graph
from scipy.spatial import cKDTree

In [ ]:
df = pd.read_pickle("data/df_all_2022_10_06.pkl")
df.shape

(1180, 95)

In [ ]:
image_path = str(df["fn_y"][0])
image_path

'/media/dimi/TOSHIBA EXT/Dimitrij Shape Masks/TMA01/12-E.tif'

In [ ]:
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

In [ ]:
image.shape

(5120, 5120)

In [ ]:
image.ndim

2

In [ ]:
len(np.unique(image))

331

In [ ]:
#| export
def show_lattice(img):
    return pd.DataFrame(img).style.set_properties(**{'font-size':'6pt'}).background_gradient('YlOrRd')

In [ ]:
#| export
def label_image(image, min_num_pixels=0):
    '''Label each cell in a TMA uniquely
       image: masked image with cell and nuclei shapes
    '''
    thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)[1]
    labels = measure.label(thresh, background=0, connectivity=image.ndim)
    mask = np.zeros(thresh.shape, dtype="uint8")

    for label in np.unique(labels): # loop over the unique components
        if label == 0: continue # if this is the background label, ignore it
        label_mask = np.zeros(thresh.shape, dtype="uint8")
        label_mask[labels == label] = label
        mask = cv2.add(mask, label_mask)
    return mask

In [ ]:
%timeit labeled_image = label_image(image)

5.98 s ± 519 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#| export
def measure_pars(labeled_image, pars_to_measure = ["label", "area", "centroid", "perimeter"], extra_measures=None):
    return pd.DataFrame(measure.regionprops_table(label_image=labeled_image,
                                        properties=pars_to_measure, 
                                        extra_properties=extra_measures))

In [ ]:
cells = measure_pars(labeled_image)

In [ ]:
cells.head()

,label,area,centroid-0,centroid-1,perimeter
0,1,1319,1141.557240,4056.058378,150.568542
1,2,1486,1180.063257,4075.713997,150.811183
2,3,1655,1204.493656,4108.811480,182.752309
3,4,1170,1229.188889,4009.226496,139.781746
4,5,1666,1266.021609,4110.200480,163.053824


In [ ]:
#| export
pd.DataFrame.cols2arr = lambda x: x.apply(lambda c: tuple(c), axis=1).apply(np.array)

In [ ]:
#| export
def dist(x, max_dist=50):
    '''x.shape: (:,2)'''
    tree = cKDTree(x)
    dists, inds  = tree.query(x, k=9, distance_upper_bound=max_dist) #lookup for 8 nn
    dists, inds = dists[:,1:], inds[:,1:]+1 #0 is index of self, label starts with 1 and not 0
    dists, inds = pd.DataFrame(dists), pd.DataFrame(inds)
    #dists, inds = pd.DataFrame(dists).applymap(lambda x: x<max_dist), pd.DataFrame(inds).applymap(lambda x: x<max_dist)    
    return dists.cols2arr(), inds.cols2arr()

In [ ]:
cells["nn_dists"], cells["nn_inds"] = dist(cells[["centroid-1","centroid-0"]])

In [ ]:
cells.head()

,label,area,centroid-0,centroid-1,perimeter,nn_inds,nn_dists
0,1,1319,1141.557240,4056.058378,150.568542,"[2, 215, 215, 215, 215, 215, 215, 215]","[43.23258854531029, inf, inf, inf, inf, inf, i..."
1,2,1486,1180.063257,4075.713997,150.811183,"[3, 1, 215, 215, 215, 215, 215, 215]","[41.13742525405137, 43.23258854531029, inf, in..."
2,3,1655,1204.493656,4108.811480,182.752309,"[2, 215, 215, 215, 215, 215, 215, 215]","[41.13742525405137, inf, inf, inf, inf, inf, i..."
3,4,1170,1229.188889,4009.226496,139.781746,"[215, 215, 215, 215, 215, 215, 215, 215]","[inf, inf, inf, inf, inf, inf, inf, inf]"
4,5,1666,1266.021609,4110.200480,163.053824,"[8, 215, 215, 215, 215, 215, 215, 215]","[39.0065974670348, inf, inf, inf, inf, inf, in..."


In [ ]:
cells["nn_dists"][0][~np.isinf(cells["nn_dists"][0])]

array([43.23258855])

In [ ]:
def find_nnn():

In [ ]:
ds,_= dist(cells[["centroid-1","centroid-0"]])

In [ ]:
ds.shape

(214, 8)

In [ ]:
ds[~np.isinf(ds,axis=1)].shape

TypeError: isinf() got an unexpected keyword argument 'axis'

In [ ]:
[list(ds[t,:])<50 for t in range(ds.shape[0])]

KeyError: (0, slice(None, None, None))

In [ ]:
(ds[ds<50])

,0,1,2,3,4,5,6,7
0,43.232589,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,41.137425,43.232589,NaN,NaN,NaN,NaN,NaN,NaN
2,41.137425,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39.006597,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
209,36.711007,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,38.416315,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,20.082581,30.708619,NaN,NaN,NaN,NaN,NaN,NaN
212,36.711007,38.416315,NaN,NaN,NaN,NaN,NaN,NaN


,0,1,2,3,4,5,6,7
0,43.232589,82.121144,99.360611,135.730418,143.716786,156.079749,160.958636,184.372691
1,41.137425,43.232589,82.667500,92.618334,113.145857,117.373388,119.745559,150.211706
2,41.137425,61.543629,82.121144,96.500951,101.284344,102.601285,112.646526,135.965167
3,56.241206,82.667500,89.858239,99.109728,99.360611,102.601285,106.328126,107.482067
4,39.006597,52.476787,61.543629,82.113914,84.358836,85.751706,92.618334,95.091566


In [ ]:
ds.applymap(lambda x: x<50)

,0,1,2,3,4,5,6,7
0,True,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
209,True,False,False,False,False,False,False,False
210,True,False,False,False,False,False,False,False
211,True,True,False,False,False,False,False,False
212,True,True,False,False,False,False,False,False
